In [1]:
import pybullet_data
import pybullet as p
import pybullet_industrial as pi
import os
import time
import numpy as np

p.connect(p.GUI)  # Use p.DIRECT for headless
#p.connect(p.DIRECT)
data_path = pybullet_data.getDataPath()
p.setAdditionalSearchPath(pybullet_data.getDataPath())
print(pybullet_data.getDataPath())
plane_id = p.loadURDF("plane.urdf")

# Get the path to the PyBullet data directory
data_path = pybullet_data.getDataPath()

# List the files in the PyBullet data directory
files = os.getcwd()
dirname = os.path.join(files,'robots')
assets = os.path.join(files,'assets')

urdf_file1 = os.path.join(dirname,'rb5_850e.urdf')
asset1 = os.path.join(assets,'specimen.urdf')
#endeffector = os.path.join(assets,'endeffector.urdf')
endeffector = os.path.join(assets,
                                'gripper_cylinder.urdf')

start_orientation = p.getQuaternionFromEuler([0, 0, 0])

cube1 = p.loadURDF("cube.urdf", np.array(
        [0, 0, 0.5]), useFixedBase=True)
robot = pi.RobotBase(urdf_file1, np.array(
        [0, 0, 1]), start_orientation)
cube2 = p.loadURDF("cube.urdf", np.array(
        [1, 0, 0.5]), useFixedBase=True)
specimen = p.loadURDF(asset1, np.array(
[1.0, 0, 1.3]), useFixedBase=True)
meshScale = [1, 1, 1]

pi.draw_robot_frames(robot, life_time=0)

tool = pi.EndeffectorTool(endeffector, np.array([2.0, 0, 1.5]), start_orientation, robot)
p.setGravity(0, 0, -9.81)

c:\Users\b34b3\AppData\Local\Programs\Python\Python312\Lib\site-packages\pybullet_data


In [2]:
def initialize():
    position_list = [1,0.16175214925291828,0.49468914613294146,1.4254185043970449,-0.350008612250527,1.5717769960272012,-0.1634941307782464,0]
    gripper_position = [0.5, 0, 1.1]
    gripper_orientation = [1, 1, 1, 1]
    #update_gripper(gripper_position, gripper_orientation)

    pose_dict = {
    "base": 0.16175214925291828,
    "shoulder": 0.49468914613294146,
    "elbow": 1.4254185043970449,
    "wrist1": -0.3500086122505276,
    "wrist2": 1.5717769960272012,
    "wrist3": -0.1634941307782464

    }

    pi.RobotBase.reset_robot(robot,[0, 0, 1], start_orientation, position_list)
    pi.RobotBase.set_joint_position(robot,pose_dict)
    for i in range(1000):
      p.stepSimulation()

def check_collision(visualize=False):
    contacts = p.getContactPoints() 
    contacts = [c for c in contacts if c[1] >= 4] # 로봇의 베이스나, 엔드이펙터와 플랜지사이의 충돌은 무시
    if contacts:
        print("TCP 링크가 무언가와 충돌 발생!")
        return True
        for contact in contacts:
            print(contact)
    
    if visualize:
        for c in contacts:
            pos_onB = c[6]
            p.removeAllUserDebugItems()
            p.addUserDebugPoints(
                pointPositions=[pos_onB],
                pointColorsRGB=[[1, 0, 0]],   # 빨간색
                pointSize=50,
                lifeTime=10
            )

def update_camera(camera_position, camera_orientation):
    camera_id = None
    for body_id in range(p.getNumBodies()):
        body_name = p.getBodyInfo(body_id)[1].decode()
        if "camera" in body_name:  # 카메라 URDF 파일명이 포함된 경우
            camera_id = body_id
            break
    pos_offset = np.array([0.1, 0, 0.04])
    ori_offset = np.array([0.5, 0.5, 0, 0])
    p.resetBasePositionAndOrientation(camera_id, camera_position + pos_offset, camera_orientation + ori_offset)

def update_specimen(specimen_position, specimen_orientation):
    specimen_id = None
    for body_id in range(p.getNumBodies()):
        body_name = p.getBodyInfo(body_id)[1].decode()
        if "my_specimen" in body_name:  # my_specimen URDF 파일명이 포함된 경우
            specimen_id = body_id
            break
    p.resetBasePositionAndOrientation(specimen_id, specimen_position, specimen_orientation)

def update_gripper(gripper_position, gripper_orientation):
    gripper_id = None
    for body_id in range(p.getNumBodies()):
        body_name = p.getBodyInfo(body_id)[1].decode()
        if "gripper" in body_name:  # gripper URDF 파일명이 포함된 경우
            gripper_id = body_id
            break
    p.resetBasePositionAndOrientation(gripper_id, gripper_position, gripper_orientation)

def show_point(tool_pos):
    p.addUserDebugPoints(
    pointPositions=[tool_pos],
    pointColorsRGB=[[1, 0, 1]],  
    pointSize=10,
    lifeTime=1
)
    
        

In [3]:
initialize()

urdf_file3 = os.path.join(assets, 'camera.urdf')
camera_parameters = {'width': 480, 'height': 240, 'fov': 60,
                        'aspect ratio': 1, 'near plane distance': 0.01, 'far plane distance': 2}
camera_orientation = p.getQuaternionFromEuler([1.57, 0, 1.57])
camera_position = tool.get_tool_pose()[0] +  np.array([0, 0, 0.1])
camera = pi.Camera(urdf_file3, camera_position,
                camera_orientation, camera_parameters)

In [4]:
base_pose = np.array([0, 0, 0.5])
set_pose = np.array([0.860, 0, 0.81])
base_orientation = np.array([1.57, 0, 1.57])

target_pose = base_pose + set_pose


specimen_position = np.array([1.0, 0, 1.30])
ring_offset = np.array([0, 0, 0.015])
specimen_orientation_euler = np.array([0, 0, 0])
specimen_orientation = p.getQuaternionFromEuler(specimen_orientation_euler)
update_specimen(specimen_position, specimen_orientation)

for _ in range(24 * 1):  # 240 Hz simulation frequency
    tool.set_tool_pose(target_pose, p.getQuaternionFromEuler(base_orientation))
    img = camera.get_image()
    p.stepSimulation()
    camera_orientation = p.getQuaternionFromEuler([1.57, 0, 1.57])
    camera_position = tool.get_tool_pose()[0] +  np.array([0, 0, 0])
    update_camera(camera_position, camera_orientation)
    show_point(tool.get_tool_pose()[0]+np.array([0.135,0,0]))
    show_point(specimen_position+ring_offset)
    #time.sleep(1./24.)

check_collision(visualize=True)

In [ ]:
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Input, Conv2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.python.client import device_lib
import gym
import numpy as np
import random as rand

LOSS_CLIPPING = 0.3

class ModelParam:
    pass

def AI_init():
    model_param = ModelParam()
    # state and action space
    model_param.state_size = 4     # [pos,vel,angle,angular_vel] 
    model_param.img_height = 240
    model_param.img_width = 480
    model_param.img_channels = 3
    model_param.action_size = 6   # [x+,x-,y+,y-,z+,z-,roll+,roll-,pitch+,pitch-,yaw+,yaw-]
    model_param.value_size = 1

    # AI hyperparameters
    model_param.learning_rate_actor = 0.0001
    model_param.learning_rate_critic = 0.0001
    model_param.epochs_cnt = 10
    model_param.discount_rate = 0.90
    model_param.smooth_rate = 0.95
    model_param.penalty = -400
    model_param.episode_num = 50
    model_param.mini_batch_step_size = 10        
    model_param.node_num = 256     # used after flatten

    # Debug variables
    model_param.moving_avg_size = 20

    # model Initialization
    model_param.model_actor = build_model_actor(model_param)
    model_param.model_critic = build_model_critic(model_param)

    # reward buffer
    model_param.reward_list= []
    model_param.count_list = []
    model_param.moving_avg_list = []

    model_param.states, model_param.states_next, model_param.action_matrixs = [],[],[]
    model_param.dones, model_param.action_probs, model_param.rewards = [],[],[]

    # dummy data
    model_param.DUMMY_ACTION_MATRIX = np.zeros((1,1,model_param.action_size))
    model_param.DUMMY_ADVANTAGE = np.zeros((1,1,model_param.value_size))
    return model_param

class MyModel(tf.keras.Model):
        def train_step(self, data):
            in_datas, out_action_probs = data
            states, action_matrixs, advantages = in_datas[0], in_datas[1], in_datas[2]
            with tf.GradientTape() as tape:
                y_pred = self([states, action_matrixs, advantages], training=True)
                new_policy = K.max(action_matrixs*y_pred, axis=-1)   
                old_policy = K.max(action_matrixs*out_action_probs, axis=-1)   
                r = new_policy/(old_policy)
                clipped = K.clip(r, 1-LOSS_CLIPPING, 1+LOSS_CLIPPING)
                loss = -K.minimum(r*advantages, clipped*advantages)
            
            trainable_vars = self.trainable_variables
            gradients = tape.gradient(loss, trainable_vars)
            self.optimizer.apply_gradients(zip(gradients, trainable_vars))
            return {"loss": loss}
        
def build_model_actor(model_param):
     # 1) The main input: an RGB image
    input_states = Input(shape=(model_param.img_height, model_param.img_width, model_param.img_channels),
                         name='input_states')

    # 2) Additional inputs (same as your old code):
    input_action_matrixs = Input(shape=(1, model_param.action_size), name='input_action_matrixs')
    input_advantages = Input(shape=(1, model_param.value_size), name='input_advantages')

    # 3) Build a small CNN
    x = Conv2D(32, kernel_size=8, strides=4, activation='relu')(input_states)
    x = Conv2D(64, kernel_size=4, strides=2, activation='relu')(x)
    x = Conv2D(64, kernel_size=3, strides=1, activation='relu')(x)
    x = Flatten()(x)
    x = Dense(model_param.node_num, activation='relu')(x)

    # 4) Output: policy over model_param.action_size
    out_actions = Dense(model_param.action_size, activation='softmax', name='output')(x)

    # 5) Create a MyModel with multi-input
    model = MyModel(inputs=[input_states, input_action_matrixs, input_advantages],
                    outputs=out_actions)
    model.compile(optimizer=Adam(learning_rate=model_param.learning_rate_actor))

    model.summary()
    return model

def build_model_critic(model_param):
    # Takes image input: (height, width, channels)
    input_states = Input(shape=(model_param.img_height, model_param.img_width, model_param.img_channels),
                         name='input_states')

    x = Conv2D(32, kernel_size=8, strides=4, activation='relu')(input_states)
    x = Conv2D(64, kernel_size=4, strides=2, activation='relu')(x)
    x = Conv2D(64, kernel_size=3, strides=1, activation='relu')(x)
    x = Flatten()(x)
    x = Dense(model_param.node_num, activation='relu')(x)

    out_values = Dense(model_param.value_size, activation='linear', name='output')(x)

    model = tf.keras.models.Model(inputs=[input_states], outputs=[out_values])
    model.compile(optimizer=Adam(learning_rate=model_param.learning_rate_critic),
                  loss='mean_squared_error')
    model.summary()
    return model

def norm(pointA, pointB):
    return np.sqrt((pointA[0] - pointB[0])**2 + (pointA[1] - pointB[1])**2 + (pointA[2] - pointB[2])**2)

def get_state(prv_states):
    # init states[4]
    dt = 1/240
    states = np.zeros(4)
    target_position = specimen_position+ring_offset
    #print(target_position)
    end_effector_position = tool.get_tool_pose()[0]+np.array([0.135,0,0])
    #print(end_effector_position)
    end_effector_orientation = p.getEulerFromQuaternion(tool.get_tool_pose()[1]) + np.array([-np.pi/2,0,-np.pi/2])
    states[0] = norm(end_effector_position,target_position) # pos diff
    states[1] = (prv_states[0]-states[0])/dt # velocity
    states[2] = norm(end_effector_orientation,specimen_orientation_euler)# angle diff
    states[3] = (prv_states[2]-states[2])/dt # angular velocity
    #print states in a row
    print("pos diff: ",states[0]," velocity: ",states[1]," angle diff: ",states[2]," angular velocity: ",states[3])
    print("angle: ",end_effector_orientation, "angle_specimen"  ,specimen_orientation_euler)
    return states

def train(model_param):
    for episode in range(model_param.episode_num):
        #initialize()
        #model_param.states = np.zeros(4)
        #model_param.states = get_state(model_param.states)
        count, reward_tot = make_memory(model_param)
        train_mini_batch(model_param)
        clear_memory(model_param)

        if count < model_param.episode_num:
            reward_tot = reward_tot-model_param.penalty

        model_param.reward_list.append(reward_tot)
        model_param.count_list.append(count)
        model_param.moving_avg_list.append(moving_avg(model_param.count_list,model_param.moving_avg_size))                
            
        if(episode % 10 == 0):
            print("episode:{}, moving_avg:{}, rewards_avg:{}".format(episode, model_param.moving_avg_list[-1], np.mean(model_param.reward_list)))
    
    model_param.save_model()
def make_memory(model_param):
        reward_tot = 0
        count = 0
        base_pose = np.array([0, 0, 0.5])
        reward = np.zeros(model_param.value_size)
        #advantage = np.zeros(model_param.value_size)
        #target = np.zeros(model_param.value_size)
        action_matrix = np.zeros(model_param.action_size)
        action_unit = np.zeros(model_param.action_size)
        done = False
        initialize()
        prev_pos_reward = 0
        while not done:
            count+=1

            #state_t = np.reshape(state,[1, 1, model_param.state_size])
            #action_matrix_t = np.reshape(action_matrix,[1, 1, model_param.action_size])

            # 1. take a picture
            img = camera.get_image()             # 카메라에서 이미지 획득
            img = img.astype(np.float32) / 255.0   # float32 변환 및 0~1 정규화
            rgb = img[:, :, :3]
            rgb = np.expand_dims(rgb, axis=0)      # (1, H, W, 3) 모양으로 확장
            model_param.states.append(rgb)
            
            # 2. calculate probabilty and predict action
            action_prob = model_param.model_actor.predict([rgb, model_param.DUMMY_ACTION_MATRIX, model_param.DUMMY_ADVANTAGE])[0]
            action = np.random.choice(model_param.action_size, 1, p=action_prob)[0]
            action_matrix = np.zeros(model_param.action_size) #초기화
            action_matrix[action] = 1 # 하나를 on 으로 만듬
            print(action)
            print(action_prob)
            
            # 3. move
            # 3.1 robot move
            action_unit = [0.05,-0.05,0.05,-0.05,0.05,-0.05] # +- 순서로 액션 을 정함
            # action 위치 외 나머지는 0으로 초기화
            action_vector = np.zeros_like(action_unit)
            action_vector[action] = action_unit[action]  # 선택된 액션만 값을 유지

            # 선택된 action에 따라 위치 업데이트 (회전은 별도로 처리)
            action_pose = np.array(tool.get_tool_pose()[0])
            if action in [0, 1]:
                # x축 변경
                action_pose[0] += action_unit[action]
            elif action in [2, 3]:
                # y축 변경
                action_pose[1] += action_unit[action]
            elif action in [4, 5]:
                # z축 변경
                action_pose[2] += action_unit[action]
            
            
            # 3.2 camera move    
            for i in range(10): # 10번 반복
                tool.set_tool_pose(action_pose, p.getQuaternionFromEuler(base_orientation)) # 액션을 적용
                camera_orientation = tool.get_tool_pose()[1] + p.getQuaternionFromEuler([1.57, 0, 1.57]) 
                camera_position = tool.get_tool_pose()[0] +  np.array([0, 0, 0])
                update_camera(camera_position, camera_orientation)    # 카메라 위치 업데이트
                p.stepSimulation()# step simulation
           #p.stepSimulation()# step simulation

            img_next = camera.get_image()  # 액션 이후 이미지 상태
            img_next_uint8 = img_next.astype(np.uint8)
            rgb_next = img_next_uint8[:, :, :3]
            rgb_next = np.expand_dims(rgb_next, axis=0)

            # 다음 이미지 상태를 states_next에 저장
            model_param.states_next.append(rgb_next)

            # 3.3 check collision
            is_collision = check_collision(visualize=False)

            # 4. calculate reward
            end_effector_position = tool.get_tool_pose()[0]+np.array([0.135,0,0])
            specimen_position = np.array([1.0, 0, 1.30])
            ring_offset = np.array([0, 0, 0.015])
            target_position = specimen_position+ring_offset
            judging_criterion = norm(end_effector_position,target_position) # pos difference
            epsilon = 1e-6
            if 1.0/(judging_criterion + epsilon) > prev_pos_reward:
                pos_reward = 1.0/(judging_criterion + epsilon)
                prev_pos_reward = pos_reward
            else:
                pos_reward = np.clip(1.0/(judging_criterion + epsilon), -10, 10)

            angle_reward = 0
            if is_collision:
                collision_reward = model_param.penalty
            else:
                collision_reward = 0

            reward = pos_reward + angle_reward + collision_reward
            print("reward: ", reward)

            # 5. judge if finished or not
            if judging_criterion < 0.02:
                reward = reward - model_param.penalty
                done = True
               
            #model_param.states.append(np.reshape(state_t, [1,model_param.state_size]))
            #model_param.states_next.append(np.reshape(state_next_t, [1,model_param.state_size]))
            model_param.action_matrixs.append(np.reshape(action_matrix, [1,model_param.action_size]))
            model_param.dones.append(np.reshape(0 if done else 1, [1,model_param.value_size]))
            model_param.action_probs.append(np.reshape(action_prob, [1,model_param.action_size]))
            model_param.rewards.append(np.reshape(reward, [1,model_param.value_size]))
            
            if(count % model_param.mini_batch_step_size == 0):
                train_mini_batch(model_param)
                clear_memory(model_param)
                initialize()

            reward_tot += reward
            
        return count, reward_tot

def make_gae(values, values_next, rewards, dones, discount_rate, smooth_rate):
    delta_adv, delta_tar, adv, target = 0, 0, 0, 0
    advantages = np.zeros(np.array(values).shape)
    targets = np.zeros(np.array(values).shape)
    for t in reversed(range(0, len(rewards))):
        delta_adv = rewards[t] + discount_rate * values_next[t] * dones[t] - values[t]
        delta_tar = rewards[t] + discount_rate * values_next[t] * dones[t]
        adv = delta_adv + smooth_rate * discount_rate * dones[t] * adv
        target = delta_tar + smooth_rate * discount_rate * dones[t] * target
        advantages[t] = adv
        targets[t] = target
    return advantages, targets


def train_mini_batch(model_param):
    if len(model_param.rewards) == 0:
        return

    # Convert collected data to numpy arrays
    action_matrixs_t = np.array(model_param.action_matrixs)
    action_probs_t = np.array(model_param.action_probs)
    rewards_t = np.array(model_param.rewards)
    dones_t = np.array(model_param.dones)

    # Use image states directly
    states_t = np.vstack(model_param.states)       # (batch_size, H, W, C)
    states_next_t = np.vstack(model_param.states_next)

    # Predict values from critic
    values = model_param.model_critic.predict(states_t)
    values_next = model_param.model_critic.predict(states_next_t)

    # Calculate advantages and targets using GAE
    advantages_t, targets_t = make_gae(values, values_next, rewards_t, dones_t,
                                       model_param.discount_rate, model_param.smooth_rate)

    # Train actor and critic
    states_t = np.array(states_t, dtype=np.float32)
    action_matrixs_t = np.array(action_matrixs_t, dtype=np.float32)
    advantages_t = np.array(advantages_t, dtype=np.float32)
    action_probs_t = np.array(action_probs_t, dtype=np.float32)

    # 입력 차원 수정
    #action_matrixs_t = np.expand_dims(action_matrixs_t, axis=1)  # (batch_size, 1, action_size)
    #advantages_t = np.expand_dims(advantages_t, axis=1)          # (batch_size, 1, value_size)

    # PPO Actor 학습
    model_param.model_actor.fit([states_t, action_matrixs_t, advantages_t], [action_probs_t],
                                epochs=model_param.epochs_cnt, verbose=0)
    model_param.model_critic.fit(states_t, targets_t, epochs=model_param.epochs_cnt, verbose=0)


def clear_memory(model_param):
    model_param.states, model_param.states_next, model_param.action_matrixs = [], [], []
    model_param.dones, model_param.action_probs, model_param.rewards = [], [], []

def moving_avg(data, size=10):
        if len(data) > size:
            c = np.array(data[len(data)-size:len(data)]) 
        else:
            c = np.array(data) 
        return np.mean(c)
    
def save_model(model_param):
    model_param.model_actor.save("./model/ppo_actor")
    model_param.model_critic.save("./model/ppo_critic")
    print("***** Training Complete and Model Saved *****")


if __name__ == "__main__":
    if 'model_param' in locals():
        del model_param

    model_param = AI_init()
    train(model_param)
    save_model(model_param)


Model: "my_model"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_states        │ (None, 240, 480,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 59, 119,   │      6,176 │ input_states[0][… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 28, 58,    │     32,832 │ conv2d[0][0]      │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 26, 56,    │     36,928 │ conv2d_1[0][0]    │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 93184)     │          0 │ conv2d_2[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 256)       │ 23,855,360 │ flatten[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_action_matri… │ (None, 1, 6)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_advantages    │ (None, 1, 1)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output (Dense)      │ (None, 6)         │      1,542 │ dense[0][0]       │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 23,932,838 (91.30 MB)

 Trainable params: 23,932,838 (91.30 MB)

 Non-trainable params: 0 (0.00 B)

...

 Trainable params: 23,931,553 (91.29 MB)

 Non-trainable params: 0 (0.00 B)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
5
[0.14738965 0.16680458 0.14483137 0.17152305 0.17178258 0.19766875]
reward:  11.59672502978044
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
5
[0.14702684 0.1678355  0.14671405 0.17207892 0.17101172 0.19533302]
reward:  9.544449804094857
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
5
[0.14770856 0.16617456 0.14759198 0.17088884 0.17230184 0.19533415]
reward:  7.790401773231707
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
2
[0.14767788 0.16588552 0.14568244 0.17153358 0.17175327 0.19746725]
reward:  7.559776082191911
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
2
[0.14812665 0.16752279 0.14524229 0.1715415  0.17112218 0.19644462]
reward:  7.008831575892479
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
5
[0.14925723 0.16653852 0.14605801 0.1717059  0.17124487 0.19519544]
reward:  6.0238434098599205
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
2
[0.14813349 0.16631718 0.145279   0.1721143  0.17050669 0.1976493 ]
reward:  5.556710874521681
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
2
[0.14850016 0

c:\Users\b34b3\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\models\functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['input_states']
Received: inputs=Tensor(shape=(10, 240, 480, 3))
  warnings.warn(msg)
c:\Users\b34b3\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\models\functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['input_states']
Received: inputs=Tensor(shape=(None, 240, 480, 3))
  warnings.warn(msg)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
2
[0.00728823 0.01013125 0.23817153 0.04085829 0.02929092 0.6742598 ]
reward:  12.174366003061625
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
5
[0.00868408 0.01196556 0.24594562 0.04475833 0.0330813  0.65556514]
reward:  10.0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
5
[0.00851231 0.01159409 0.24418922 0.0441459  0.03257785 0.6589806 ]
reward:  9.164853899083207
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
5
[0.00861056 0.01167234 0.24392617 0.04447377 0.03235601 0.65896124]
reward:  7.576152814275589
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
5
[0.0084083  0.0115296  0.2432415  0.04405533 0.03204433 0.660721  ]
reward:  6.336224517125618
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
5
[0.00829035 0.01140194 0.24273887 0.04383542 0.03175819 0.6619753 ]
reward:  5.394541946783456
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
2
[0.00825857 0.01135355 0.24255301 0.04385383 0.0317929  0.6621882 ]
reward:  5.1742252343850925
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
5
[0.00828601 0.01139937 0.2

In [ ]:
action_unit = np.zeros(model_param.action_size) # 12
action_unit= [1,1,1,1,1,1,1,1,1,1,1,1] 

initialize()
base_pose = np.array([0, 0, 0.5])
action_pose = tool.get_tool_pose()[0] 
for _ in range(24 * 2):
    tool.set_tool_pose(action_pose, p.getQuaternionFromEuler(base_orientation))
    p.stepSimulation()

camera_orientation = p.getQuaternionFromEuler([1.57, 0, 1.57])
camera_position = tool.get_tool_pose()[0] +  np.array([0, 0, 0])
update_camera(camera_position, camera_orientation)

In [ ]:
 tool.get_tool_pose()[1]

array([0.50199083, 0.48733655, 0.51342403, 0.49689441])

In [ ]:
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Input, Dense, Flatten
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.python.client import device_lib
import gym
import numpy as np
import random as rand

LOSS_CLIPPING = 0.2
class Agent(object):
    def __init__(self):
        self.env = gym.make('CartPole-v1')
        self.state_size = self.env.observation_space.shape[0]
        self.action_size = self.env.action_space.n
        self.value_size = 1
        
        self.node_num = 24
        self.learning_rate_actor = 0.0005
        self.learning_rate_critic = 0.0005
        self.epochs_cnt = 5
        self.model_actor = self.build_model_actor()
        self.model_critic = self.build_model_critic()
        
        self.discount_rate = 0.98
        self.smooth_rate = 0.95
        self.penalty = -400
        
        self.episode_num = 500
        self.mini_batch_step_size = 32        
        
        self.moving_avg_size = 20
        self.reward_list= []
        self.count_list = []
        self.moving_avg_list = []
        
        self.states, self.states_next, self.action_matrixs = [],[],[]
        self.dones, self.action_probs, self.rewards = [],[],[]
        self.DUMMY_ACTION_MATRIX = np.zeros((1,1,self.action_size))
        self.DUMMY_ADVANTAGE = np.zeros((1,1,self.value_size))
        
    class MyModel(tf.keras.Model):
        def train_step(self, data):
            in_datas, out_action_probs = data
            states, action_matrixs, advantages = in_datas[0], in_datas[1], in_datas[2]
            with tf.GradientTape() as tape:
                y_pred = self(states, training=True)
                new_policy = K.max(action_matrixs*y_pred, axis=-1)   
                old_policy = K.max(action_matrixs*out_action_probs, axis=-1)   
                r = new_policy/(old_policy)
                clipped = K.clip(r, 1-LOSS_CLIPPING, 1+LOSS_CLIPPING)
                loss = -K.minimum(r*advantages, clipped*advantages)
            
            trainable_vars = self.trainable_variables
            gradients = tape.gradient(loss, trainable_vars)
            self.optimizer.apply_gradients(zip(gradients, trainable_vars))
            
    def build_model_actor(self):
        input_states = Input(shape=(1,self.state_size), name='input_states')
        input_action_matrixs = Input(shape=(1,self.action_size), name='input_action_matrixs')
        input_advantages = Input(shape=(1,self.value_size), name='input_advantages')

        x = (input_states)
        x = Dense(self.node_num, activation='relu')(x)
        out_actions = Dense(self.action_size, activation='softmax', name='output')(x)
        
        model = self.MyModel(inputs=[input_states, input_action_matrixs, input_advantages], outputs=out_actions)
        model.compile(optimizer=Adam(learning_rate=self.learning_rate_actor))
        
        model.summary()
        return model
    
    def build_model_critic(self):
        input_states = Input(shape=(1,self.state_size), name='input_states')
        x = (input_states)
        x = Dense(self.node_num, activation='relu')(x)
        out_values = Dense(self.value_size, activation='linear', name='output')(x)
        
        model = tf.keras.models.Model(inputs=[input_states], outputs=[out_values])
        model.compile(optimizer=Adam(learning_rate=self.learning_rate_critic),
                      loss='mean_squared_error'
                     )
        model.summary()
        return model

    def train(self):
        for episode in range(self.episode_num):

            state = self.env.reset()
            state = state[0]
            self.env.max_episode_steps = 500

            count, reward_tot = self.make_memory(episode, state)
            self.train_mini_batch()
            self.clear_memory()
            
            if count < 500:
                reward_tot = reward_tot-self.penalty
                
            self.reward_list.append(reward_tot)
            self.count_list.append(count)
            self.moving_avg_list.append(self.moving_avg(self.count_list,self.moving_avg_size))                
            
            if(episode % 10 == 0):
                print("episode:{}, moving_avg:{}, rewards_avg:{}".format(episode, self.moving_avg_list[-1], np.mean(self.reward_list)))
        self.save_model()
        
    def make_memory(self, episode, state):
        reward_tot = 0
        count = 0
        reward = np.zeros(self.value_size)
        advantage = np.zeros(self.value_size)
        target = np.zeros(self.value_size)
        action_matrix = np.zeros(self.action_size)
        done = False
        
        while not done:
            count+=1

            state_t = np.reshape(state,[1, 1, self.state_size])
            action_matrix_t = np.reshape(action_matrix,[1, 1, self.action_size])
            
            action_prob = self.model_actor.predict([state_t, self.DUMMY_ACTION_MATRIX, self.DUMMY_ADVANTAGE])

            action = np.random.choice(self.action_size, 1, p=action_prob[0][0])[0]
            action_matrix = np.zeros(self.action_size) #초기화
            action_matrix[action] = 1

            state_next, reward, done, none, none2 = self.env.step(action)
            
            state_next_t = np.reshape(state_next,[1, 1, self.state_size])
            
            if count < 500 and done:
                reward = self.penalty 
        
            self.states.append(np.reshape(state_t, [1,self.state_size]))
            self.states_next.append(np.reshape(state_next_t, [1,self.state_size]))
            self.action_matrixs.append(np.reshape(action_matrix, [1,self.action_size]))
            self.dones.append(np.reshape(0 if done else 1, [1,self.value_size]))
            self.action_probs.append(np.reshape(action_prob, [1,self.action_size]))
            self.rewards.append(np.reshape(reward, [1,self.value_size]))
            
            if(count % self.mini_batch_step_size == 0):
                self.train_mini_batch()
                self.clear_memory()

            reward_tot += reward
            state = state_next
            
        return count, reward_tot
        
    def make_gae(self, values, values_next, rewards, dones):
        delta_adv, delta_tar, adv, target = 0, 0, 0, 0
        advantages = np.zeros(np.array(values).shape)
        targets = np.zeros(np.array(values).shape)
        for t in reversed(range(0, len(rewards))):
            delta_adv = rewards[t] + self.discount_rate * values_next[t] * dones[t] - values[t]
            delta_tar = rewards[t] + self.discount_rate * values_next[t] * dones[t]
            adv = delta_adv + self.smooth_rate*self.discount_rate * dones[t] * adv
            target = delta_tar + self.smooth_rate*self.discount_rate * dones[t] * target
            advantages[t] = adv
            targets[t] = target
        return advantages, targets

    def train_mini_batch(self):
        
        if len(self.states) == 0:
            return
        
        states_t = np.array(self.states)
        states_next_t = np.array(self.states_next)
        action_matrixs_t = np.array(self.action_matrixs)
        action_probs_t = np.array(self.action_probs)
        rewards_t = np.array(self.rewards)

        values = self.model_critic.predict(states_t)
        values_next = self.model_critic.predict(states_next_t)
        
        advantages, targets = self.make_gae(values, values_next, self.rewards, self.dones)
        advantages_t = np.array(advantages)
        targets_t = np.array(targets)

        self.model_actor.fit([states_t, action_matrixs_t, advantages_t], [action_probs_t], epochs=self.epochs_cnt, verbose=0)
        self.model_critic.fit(states_t, targets_t, epochs=self.epochs_cnt, verbose=0)       
 
    def clear_memory(self):
        self.states, self.states_next, self.action_matrixs = [],[],[]
        self.dones, self.action_probs, self.rewards = [],[],[]
        
    def moving_avg(self, data, size=10):
        if len(data) > size:
            c = np.array(data[len(data)-size:len(data)]) 
        else:
            c = np.array(data) 
        return np.mean(c)
    
    def save_model(self):
        self.model_actor.save("./model/ppo")
        print("*****end learing")

if __name__ == "__main__":
    agent = Agent()
    agent.train()

Model: "my_model_11"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_states        │ (None, 1, 4)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_22 (Dense)    │ (None, 1, 24)     │        120 │ input_states[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_action_matri… │ (None, 1, 2)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_advantages    │ (None, 1, 1)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output (Dense)      │ (None, 1, 2)      │         50 │ dense_22[0][0]    │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 170 (680.00 B)

 Trainable params: 170 (680.00 B)

 Non-trainable params: 0 (0.00 B)

...

 Total params: 145 (580.00 B)

 Trainable params: 145 (580.00 B)

 Non-trainable params: 0 (0.00 B)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step


AttributeError: module 'numpy' has no attribute 'bool8'

In [ ]:
env = gym.make('CartPole-v1')
env.observation_space.shape[0] # 위치, 속도, 각도, 각속도?
env.action_space.n # 좌우 2개

2

In [ ]:
action = 1
action_unit = [0.01,-0.01,0.01,-0.01,0.01,-0.01] # +- 순서로 액션 을 정함

action_pose = np.array(tool.get_tool_pose()[0])
if action in [0, 1]:
    # x축 변경

    action_pose[0] += action_unit[action]
elif action in [2, 3]:
    # y축 변경
    action_pose[1] += action_unit[action]
elif action in [4, 5]:
    # z축 변경
    action_pose[2] += action_unit[action]

print(action_pose)

[0.79932191 0.03632879 1.39198408]


In [ ]:
if 'state' not in locals():
    state =np.zeros(4)

state = get_state(state)


# if variable state does not exsit


pos diff:  0.16357298126706452  velocity:  0.0  angle diff:  0.02465193761612129  angular velocity:  0.0
angle:  [-0.02178325 -0.01130558 -0.00232204] angle_specimen [0 0 0]
